## Load Dataset

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
# libgomp issue, must import n2 before torch
from n2 import HnswIndex

In [4]:
import sys

sys.path.insert(0, '..')

In [5]:
import os
home_dir = os.getenv('HOME')

https://dbs.uni-leipzig.de/research/projects/object_matching/benchmark_datasets_for_entity_resolution

In [6]:
import glob
import csv
from tqdm.auto import tqdm

from entity_embed.data_utils.utils import Enumerator

row_dict = {}
id_enumerator = Enumerator()
left_id_set = set()
right_id_set = set()
rows_total = 1363 + 3226
clusters_total = 1300

with tqdm(total=rows_total) as pbar:
    with open(f'{home_dir}/Downloads/amazon_google_exp_data/tableA.csv') as f:
        for row in csv.DictReader(f):
            row['id'] = id_enumerator[f'left-{int(row["id"])}']
            row['source'] = 'left'
            row_dict[row['id']] = row
            left_id_set.add(row['id'])
            pbar.update(1)
    
    with open(f'{home_dir}/Downloads/amazon_google_exp_data/tableB.csv') as f:
        for row in csv.DictReader(f):
            row['id'] = id_enumerator[f'right-{int(row["id"])}']
            row['source'] = 'right'
            row_dict[row['id']] = row
            right_id_set.add(row['id'])
            pbar.update(1)

In [7]:
train_true_pos_pair_set = set()
train_true_neg_pair_set = set()
valid_true_pos_pair_set = set()
valid_true_neg_pair_set = set()
test_true_pos_pair_set = set()
test_true_neg_pair_set = set()

with open(f'{home_dir}/Downloads/amazon_google_exp_data/train.csv') as f:
    for row in csv.DictReader(f):
        id_left = id_enumerator[f'left-{int(row["ltable_id"])}']
        id_right = id_enumerator[f'right-{int(row["rtable_id"])}']
        if int(row['label']) == 1:
            train_true_pos_pair_set.add((id_left, id_right))
        else:
            train_true_neg_pair_set.add((id_left, id_right))

with open(f'{home_dir}/Downloads/amazon_google_exp_data/valid.csv') as f:
    for row in csv.DictReader(f):
        id_left = id_enumerator[f'left-{int(row["ltable_id"])}']
        id_right = id_enumerator[f'right-{int(row["rtable_id"])}']
        if int(row['label']) == 1:
            valid_true_pos_pair_set.add((id_left, id_right))
        else:
            valid_true_neg_pair_set.add((id_left, id_right))

with open(f'{home_dir}/Downloads/amazon_google_exp_data/test.csv') as f:
    for row in csv.DictReader(f):
        id_left = id_enumerator[f'left-{int(row["ltable_id"])}']
        id_right = id_enumerator[f'right-{int(row["rtable_id"])}']
        if int(row['label']) == 1:
            test_true_pos_pair_set.add((id_left, id_right))
        else:
            test_true_neg_pair_set.add((id_left, id_right))
        
display(('train_true_pos_pair_set', len(train_true_pos_pair_set)))
display(('train_true_neg_pair_set', len(train_true_neg_pair_set)))
display(('valid_true_pos_pair_set', len(valid_true_pos_pair_set)))
display(('valid_true_neg_pair_set', len(valid_true_neg_pair_set)))
display(('test_true_pos_pair_set', len(test_true_pos_pair_set)))
display(('test_true_neg_pair_set', len(test_true_neg_pair_set)))

('train_true_pos_pair_set', 699)

('train_true_neg_pair_set', 6175)

('valid_true_pos_pair_set', 234)

('valid_true_neg_pair_set', 2059)

('test_true_pos_pair_set', 234)

('test_true_neg_pair_set', 2059)

## Preprocess

In [8]:
attr_list = ['title', 'manufacturer', 'price']

In [9]:
import unidecode
import itertools
from entity_embed import default_tokenizer

def clean_str(s):
    s = unidecode.unidecode(s).lower().strip()
    s_tokens = itertools.islice((s_part[:30] for s_part in default_tokenizer(s)), 0, 30)
    return ' '.join(s_tokens)[:300]

for row in tqdm(row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

## Init Data Module

In [10]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [11]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [12]:
attr_info_dict = {
    'title': {
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
    },
    'manufacturer': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'price': {
        'field_type': "STRING",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    }
}

In [13]:
from entity_embed import build_row_numericalizer

row_numericalizer = build_row_numericalizer(attr_info_dict, row_dict=row_dict)
row_numericalizer.attr_info_dict

16:34:35 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
16:34:39 INFO:For attr='manufacturer', computing actual alphabet and max_str_len
16:34:39 INFO:actual_max_str_len=15 must be pair to enable NN pooling. Updating to 16
16:34:39 INFO:For attr='manufacturer', using actual_max_str_len=16
16:34:39 INFO:For attr='price', computing actual alphabet and max_str_len
16:34:39 INFO:actual_max_str_len=11 must be pair to enable NN pooling. Updating to 12
16:34:39 INFO:For attr='price', using actual_max_str_len=12


{'title': NumericalizeInfo(field_type=<FieldType.SEMANTIC_MULTITOKEN: 'semantic_multitoken'>, tokenizer=<function default_tokenizer at 0x7f50d980dca0>, alphabet=None, max_str_len=None, vocab=<torchtext.vocab.Vocab object at 0x7f50b15a9ca0>),
 'manufacturer': NumericalizeInfo(field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer=<function default_tokenizer at 0x7f50d980dca0>, alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=16, vocab=None),
 'price': NumericalizeInfo(field_type=<FieldType.STRING: 'string'>, tokenizer=<function default_tokenizer at 0x7f50d980dca0>, alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 

In [14]:
from entity_embed import PairwiseDataModule

datamodule = PairwiseDataModule(
    row_dict=row_dict,
    row_numericalizer=row_numericalizer,
    pair_batch_size=50,
    row_batch_size=16,
    train_true_pos_pair_set=train_true_pos_pair_set,
    train_true_neg_pair_set=train_true_neg_pair_set,
    valid_true_pos_pair_set=valid_true_pos_pair_set,
    valid_true_neg_pair_set=valid_true_neg_pair_set,
    test_true_pos_pair_set=test_true_pos_pair_set,
    test_true_neg_pair_set=test_true_neg_pair_set,
    random_seed=random_seed
)

## Training

In [15]:
from entity_embed import PairwiseLinkageEmbed

ann_k = 100
model = PairwiseLinkageEmbed(
    datamodule,
    ann_k=ann_k,
    use_mask=True,
    embedding_size=300
)

In [16]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 50
early_stop_callback = EarlyStopping(
   monitor='valid_f1_at_0.7',
   min_delta=0.00,
   patience=10,
   verbose=True,
   mode='max'
)
tb_log_dir = 'tb_logs'
tb_name = 'amzn-googl'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_log_dir, name=tb_name)
)

16:34:39 INFO:GPU available: True, used: True
16:34:39 INFO:TPU available: None, using: 0 TPU cores
16:34:39 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [17]:
trainer.fit(model, datamodule)

16:34:41 INFO:
  | Name        | Type                | Params
----------------------------------------------------
0 | blocker_net | BlockerNet          | 4.8 M 
1 | losser      | CosineEmbeddingLoss | 0     
----------------------------------------------------
3.1 M     Trainable params
1.7 M     Non-trainable params
4.8 M     Total params


1

In [18]:
model.blocker_net.get_signature_weights()

{'title': 0.45444154739379883,
 'manufacturer': 0.2673819661140442,
 'price': 0.2781706750392914}

## Testing

In [21]:
trainer.test(ckpt_path='best')


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_f1_at_0.3': 0.0031102131958105164,
 'test_f1_at_0.5': 0.0031102131958105164,
 'test_f1_at_0.7': 0.0031141868512110727,
 'test_f1_at_0.9': 0.004721282243520902,
 'test_pair_entity_ratio_at_0.3': 80.72971520687803,
 'test_pair_entity_ratio_at_0.5': 80.72971520687803,
 'test_pair_entity_ratio_at_0.7': 80.62654486835035,
 'test_pair_entity_ratio_at_0.9': 52.91133799032778,
 'test_precision_at_0.3': 0.0015575287210958613,
 'test_precision_at_0.5': 0.0015575287210958613,
 'test_precision_at_0.7': 0.0015595217466643563,
 'test_precision_at_0.9': 0.0023662509647804364,
 'test_recall_at_0.3': 1.0,
 'test_recall_at_0.5': 1.0,
 'test_recall_at_0.7': 1.0,
 'test_recall_at_0.9': 0.9957264957264957}
--------------------------------------------------------------------------------


[{'test_precision_at_0.3': 0.0015575287210958613,
  'test_recall_at_0.3': 1.0,
  'test_f1_at_0.3': 0.0031102131958105164,
  'test_pair_entity_ratio_at_0.3': 80.72971520687803,
  'test_precision_at_0.5': 0.0015575287210958613,
  'test_recall_at_0.5': 1.0,
  'test_f1_at_0.5': 0.0031102131958105164,
  'test_pair_entity_ratio_at_0.5': 80.72971520687803,
  'test_precision_at_0.7': 0.0015595217466643563,
  'test_recall_at_0.7': 1.0,
  'test_f1_at_0.7': 0.0031141868512110727,
  'test_pair_entity_ratio_at_0.7': 80.62654486835035,
  'test_precision_at_0.9': 0.0023662509647804364,
  'test_recall_at_0.9': 0.9957264957264957,
  'test_f1_at_0.9': 0.004721282243520902,
  'test_pair_entity_ratio_at_0.9': 52.91133799032778}]

## Testing manually 

In [20]:
from entity_embed.data_utils.utils import id_pairs_to_cluster_mapping_and_dict, cluster_dict_to_filtered_row_dict

__, test_cluster_dict = id_pairs_to_cluster_mapping_and_dict(test_true_pair_set)
test_row_dict = cluster_dict_to_filtered_row_dict(row_dict, test_cluster_dict)
test_left_vector_dict, test_right_vector_dict = model.predict(
    row_dict=test_row_dict,
    left_id_set=left_id_set,
    right_id_set=right_id_set,
    batch_size=16
)

NameError: name 'test_true_pair_set' is not defined

In [ ]:
embedding_size = model.blocker_net.embedding_size

In [ ]:
assert (len(test_left_vector_dict) + len(test_right_vector_dict)) == len(test_row_dict)

In [ ]:
%%time

from entity_embed import ANNLinkageIndex

ann_index = ANNLinkageIndex(embedding_size=embedding_size)
ann_index.insert_vector_dict(left_vector_dict=test_left_vector_dict, right_vector_dict=test_right_vector_dict)
ann_index.build()

In [ ]:
%%time

sim_threshold = 0.7
found_pair_set = ann_index.search_pairs(
    k=ann_k,
    sim_threshold=sim_threshold,
    left_vector_dict=test_left_vector_dict,
    right_vector_dict=test_right_vector_dict,
)

In [ ]:
from entity_embed.evaluation import pair_entity_ratio

pair_entity_ratio(len(found_pair_set), len(test_row_dict))

In [ ]:
from entity_embed.evaluation import precision_and_recall, f1_score

precision, recall = precision_and_recall(found_pair_set, test_true_pair_set)
precision, recall

In [ ]:
f1_score(precision, recall)

In [ ]:
false_positives = list(found_pair_set - test_true_pair_set)
len(false_positives)

In [ ]:
false_negatives = list(test_true_pair_set - found_pair_set)
len(false_negatives)

In [ ]:
cos_similarity = lambda a, b: np.dot(a, b)

In [ ]:
for (id_left, id_right) in false_negatives[:10]:
    display(
        (
            cos_similarity(test_left_vector_dict[id_left], test_right_vector_dict[id_right]),
            row_dict[id_left], row_dict[id_right]
        )
    )